In [ ]:
path = "/home/ljiahao/xianglin/git_space/SafetyReasoningDataEvol/data/processed/circuitbreaker_train_yes.json"

import json
with open(path, "r") as f:
    data = json.load(f)

data[0]

In [ ]:
from src.evol.answers import adapt_scot_string_two_stage, extract_yes_categories
evolved_answer = data[0]['evolved_answer']

adapted_output = adapt_scot_string_two_stage(evolved_answer)
# filter out full No answer
extracted_categories = extract_yes_categories(adapted_output)
print("Length of extracted categories:", len(extracted_categories))
print("Adapted output:", adapted_output)


In [23]:
for item in data:
    evolved_answer = item['evolved_answer']
    adapted_output = adapt_scot_string_two_stage(evolved_answer)
    item['evolved_answer_modified'] = adapted_output


In [24]:
# save
with open(path, 'w') as f:
    json.dump(data, f, indent=4)

In [ ]:
import json
dolly_path = "/home/ljiahao/xianglin/git_space/SafetyReasoningDataEvol/data/processed/dolly_val_intention.json"

with open(dolly_path, 'r') as f:
    dolly = json.load(f)

dolly[0]

In [ ]:
evolved_answer = dolly[0]['evolved_answer']
intention = dolly[0]['intention'].split("The intent of the text is to ")[-1].split(".")[0]
print(evolved_answer)

In [ ]:
from src.evol.answers import adapt_scot_string_two_stage, extract_yes_categories
adapted_output = adapt_scot_string_two_stage(evolved_answer, intention)
# filter out full No answer
extracted_categories = extract_yes_categories(adapted_output)
print("Length of extracted categories:", len(extracted_categories))
print("Adapted output:", adapted_output)

In [6]:
new_dolly = []
for item in dolly:
    evolved_answer = item['evolved_answer']
    intention = item['intention']
    if type(intention) == list:
        continue
    intention = intention.split("The intent of the text is to ")[-1].split(".")[0]
    adapted_output = adapt_scot_string_two_stage(evolved_answer, intention)
    item['evolved_answer_modified'] = adapted_output
    new_dolly.append(item)


In [7]:
# save 
with open(dolly_path, 'w') as f:
    json.dump(new_dolly, f, indent=4)

